In [1]:
from src.datasets import AugmentedSpeechDataset
from src.utils import checkpoint
from src.utils.notebook import Sampler, visualize_audio 

In [2]:
dataset = AugmentedSpeechDataset(train=False, quiet=True)

In [3]:
# CHECKPOINT = 'wave-u-net-new-baseline-1573165774.full.ckpt'  # GAN + FL baseline 6 epochs
# CHECKPOINT = 'wave-u-net-mse-gan-feature-losses-1573190574.full.ckpt' # GAN + FL + MSE 6 epochs 
CHECKPOINT = 'wave-u-net-mse-onlyy-1573191688.full.ckpt' # MSE loss only, 18 epochs
# CHECKPOINT = 'wave-u-net-feature-loss-only-1573192123.full.ckpt' # FL only, 12 epochs

In [4]:
net = checkpoint.load(CHECKPOINT, use_cuda=False)
net = net.eval()
sampler = Sampler(net, dataset)

In [5]:
# DOWNLOAD = '/home/matt/Downloads/indicator.resampled.wav'
# noisy_arr, pred_clean, pred_noise = get_results_from_file(DOWNLOAD, start=0)#2**18)

In [6]:
noisy_arr, clean_arr, pred_clean, pred_noise, true_noise = sampler.get_results(2)

NameError: name 'torch' is not defined

In [7]:
visualize_audio(pred_clean, 'Predicted clean signal')

NameError: name 'pred_clean' is not defined

In [8]:
visualize_audio(clean_arr, 'Clean signal')

NameError: name 'clean_arr' is not defined

In [9]:
visualize_audio(noisy_arr, 'Noisy signal')

NameError: name 'noisy_arr' is not defined

In [10]:
visualize_audio(pred_noise, 'Predicted noise')

NameError: name 'pred_noise' is not defined

In [11]:
visualize_audio(true_noise, 'True noise signal')

NameError: name 'true_noise' is not defined